<a href="https://colab.research.google.com/github/MianzhiHu/Propofol/blob/master/mixed_effect_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages(c("lme4", "optimx", "lmerTest", "emmeans", "effects", "pbkrtest"))

In [ ]:
library(dplyr) 
library(ggplot2)
library(tidyverse)
library(lme4)
library(optimx)
library(emmeans)
library(lmerTest)
library(effects)

In [ ]:
x <- read.csv("/content/hurst_averages.csv") %>%
  separate(key, into = c("subject","nan1", "condition", "sedation_level", "nan2"), convert = TRUE, sep = "_") %>%
  select("subject", "condition", "sedation_level", "hurst_average", "r_squared") %>%
  mutate(sedation_level = as.character(sedation_level)) %>%
  mutate(sedation_level = fct_recode(sedation_level,
                                "Awake" = "1",
                                "Mild" = "2",
                                "Deep" = "3",
                                "Recovery" = "4")) %>%
  filter(r_squared > 0.9)
x$subject <- factor(x$subject)
x$sedation_level <- factor(x$sedation_level, levels=c("Awake","Mild", "Deep", "Recovery")) 
x$condition <- factor(x$condition, levels=c("rest","movie")) 
 

contrasts(x$sedation_level) = contr.poly(4)
options(contrasts=c("contr.sum","contr.poly"))

# Normalize the data
x$hurst <- scale(x$hurst_average) 

x

In [ ]:
y <- read.csv("/content/ss_out_review_compiled_ANAT.csv") %>%
  rename("subject" = "sub",
          "condition" = "task",
          "sedation_level" = "run") %>%
  mutate(sedation_level = as.character(sedation_level)) %>%
  mutate(sedation_level = fct_recode(sedation_level,
                                "Awake" = "1",
                                "Mild" = "2",
                                "Deep" = "3",
                                "Recovery" = "4")) %>%
  full_join(x, by = c("subject", "sedation_level", "condition"))

ggplot(data = y) +
  geom_line(aes(x = motion, y = r_squared)) +
  geom_smooth(aes(x = motion, y = r_squared))

y

In [ ]:
z <- y %>%
  filter(r_squared > 0.9)

z

In [ ]:
mixed_effect_total = lmer(hurst ~ sedation_level + (1 | subject) + (1 | condition), data=z)
summary(mixed_effect_total)

In [ ]:
anova(mixed_effect_total)
plot(allEffects(mixed_effect_total))
ranef(mixed_effect_total)$subject
emmeans(mixed_effect_total, list(pairwise ~ sedation_level), adjust = "tukey")

In [ ]:
mixed_effect_condition = lmer(hurst ~ condition + (1 | subject) + (1 | sedation_level), data=z)
summary(mixed_effect_condition)
plot(allEffects(mixed_effect_condition))

In [ ]:
movie <- z %>%
  filter(condition == "movie")

mixed_effect_movie = lmer(hurst ~ sedation_level + (1 | subject), data=movie)
summary(mixed_effect_movie)
anova(mixed_effect_movie)
plot(allEffects(mixed_effect_movie))
emmeans(mixed_effect_movie, list(pairwise ~ sedation_level), adjust = "tukey")

In [ ]:
rest <- z %>%
  filter(condition == "rest")

mixed_effect_rest = lmer(hurst ~ sedation_level + (1 | subject), data=rest)
summary(mixed_effect_rest)
anova(mixed_effect_rest)
plot(allEffects(mixed_effect_rest))
emmeans(mixed_effect_rest, list(pairwise ~ sedation_level), adjust = "tukey")

In [ ]:
mixed_effect = lmer(hurst ~ sedation_level*condition + (1 | subject), data=z)
summary(mixed_effect)
anova(mixed_effect)
plot(allEffects(mixed_effect))
emmeans(mixed_effect, list(pairwise ~ sedation_level*condition), adjust = "tukey")

In [ ]:
rest_100 <- read.csv("/content/rest_100.csv") %>%
  separate(col = X, into = c("subject"), convert = TRUE, sep = "_") %>%
  mutate(recovery_early = rowMeans(across(X0:X9), na.rm = TRUE)) %>%
  mutate(recovery_late = rowMeans(across(X36:X45), na.rm = TRUE)) %>%
  mutate(condition = "rest") %>%
  select("subject", "recovery_early" ,"recovery_late", "condition") %>%
  pivot_longer(c("recovery_early", "recovery_late"), names_to = "sedation_level", values_to = "hurst_average") %>%
  mutate(hurst = scale(hurst_average)) %>%
  select("subject", "sedation_level", "hurst", "condition") %>%
  full_join(x, by = c("subject", "sedation_level", "hurst", "condition")) %>%
  filter(condition == "rest") %>%
  filter(sedation_level != "Recovery")

rest_100$subject <- factor(rest_100$subject)
rest_100$sedation_level <- factor(rest_100$sedation_level, levels=c("Awake","Mild", "Deep", "recovery_early", "recovery_late"))

contrasts(rest_100$sedation_level) = contr.poly(5)
options(contrasts=c("contr.sum","contr.poly"))

rest_100

In [ ]:
mixed_effect_rest_100 = lmer(hurst ~ sedation_level + (1 | subject), data=rest_100)
summary(mixed_effect_rest_100)
anova(mixed_effect_rest_100)
plot(allEffects(mixed_effect_rest_100))
emmeans(mixed_effect_rest_100, list(pairwise ~ sedation_level), adjust = "tukey")

In [ ]:
movie_100 <- read.csv("/content/movie_100.csv") %>%
  separate(col = X, into = c("subject"), convert = TRUE, sep = "_") %>%
  mutate(recovery_early = rowMeans(across(X0:X19), na.rm = TRUE)) %>%
  mutate(recovery_late = rowMeans(across(X46:X65), na.rm = TRUE)) %>%
  mutate(condition = "movie") %>%
  select("subject", "recovery_early" ,"recovery_late", "condition") %>%
  pivot_longer(c("recovery_early", "recovery_late"), names_to = "sedation_level", values_to = "hurst_average") %>%
  mutate(hurst = scale(hurst_average)) %>%
  select("subject", "sedation_level", "hurst", "condition") %>%
  full_join(x, by = c("subject", "sedation_level", "hurst", "condition")) %>%
  filter(condition == "movie") %>%
  filter(sedation_level != "Recovery")

movie_100$subject <- factor(movie_100$subject)
movie_100$sedation_level <- factor(movie_100$sedation_level, levels=c("Awake","Mild", "Deep", "recovery_early", "recovery_late"))

contrasts(movie_100$sedation_level) = contr.poly(5)
options(contrasts=c("contr.sum","contr.poly"))

movie_100

In [ ]:
mixed_effect_movie_100 = lmer(hurst ~ sedation_level + (1 | subject), data=movie_100)
summary(mixed_effect_movie_100)
anova(mixed_effect_movie_100)
plot(allEffects(mixed_effect_movie_100))
emmeans(mixed_effect_movie_100, list(pairwise ~ sedation_level), adjust = "tukey")

In [ ]:
total_100 <- rest_100 %>%
  full_join(movie_100)

total_100$subject <- factor(total_100$subject)
total_100$sedation_level <- factor(total_100$sedation_level, levels=c("Awake","Mild", "Deep", "recovery_early", "recovery_late"))

contrasts(total_100$sedation_level) = contr.poly(5)
options(contrasts=c("contr.sum","contr.poly"))

total_100

In [ ]:
mixed_effect_total_100 = lmer(hurst ~ sedation_level*condition + (1 | subject), data=total_100)
summary(mixed_effect_total_100)
anova(mixed_effect_total_100)
plot(allEffects(mixed_effect_total_100))
emmeans(mixed_effect_total_100, list(pairwise ~ sedation_level*condition), adjust = "tukey")